# Product Sales Prediction

In [19]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, f1_score

In [20]:
data = pd.read_csv('SalesKaggle3.csv')

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198917 entries, 0 to 198916
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Order             198917 non-null  int64  
 1   File_Type         198917 non-null  object 
 2   SKU_number        198917 non-null  int64  
 3   SoldFlag          75996 non-null   float64
 4   SoldCount         75996 non-null   float64
 5   MarketingType     198917 non-null  object 
 6   ReleaseNumber     198917 non-null  int64  
 7   New_Release_Flag  198917 non-null  int64  
 8   StrengthFactor    198917 non-null  float64
 9   PriceReg          198917 non-null  float64
 10  ReleaseYear       198917 non-null  int64  
 11  ItemCount         198917 non-null  int64  
 12  LowUserPrice      198917 non-null  float64
 13  LowNetPrice       198917 non-null  float64
dtypes: float64(6), int64(6), object(2)
memory usage: 21.2+ MB


## Preprocessing

In [22]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Only use historical data
    df = df.query("File_Type == 'Historical'")
    
    # Drop unused columns
    df = df.drop(['Order', 'File_Type', 'SKU_number', 'SoldCount'], axis=1)
    
    # Shuffle data
    df = df.sample(frac=1.0, random_state=1)
    
    # Split df into X and y
    y = df['SoldFlag']
    X = df.drop('SoldFlag', axis=1)
    
    return X, y

In [23]:
X, y = preprocess_inputs(data)

In [24]:
X

,MarketingType,ReleaseNumber,New_Release_Flag,StrengthFactor,PriceReg,ReleaseYear,ItemCount,LowUserPrice,LowNetPrice
37862,S,12,1,545082.0,96.67,2011,12,73.74,101.33
35304,S,2,1,4273940.0,58.00,2002,32,85.60,23.98
26138,D,9,1,165834.0,76.95,2011,48,75.57,42.67
52327,S,22,1,79220.0,54.25,2012,31,36.47,22.49
6038,D,8,1,80014.0,38.99,2008,62,153.24,69.43
...,...,...,...,...,...,...,...,...,...
20609,D,8,1,40841.0,103.24,2010,48,99.50,115.55
21440,D,0,0,1611172.0,86.64,2011,19,55.19,78.38
73349,S,2,1,1628317.0,69.99,2004,43,4.02,30.43
50057,S,2,1,1660915.0,44.00,2004,32,34.51,10.12


In [25]:
y

37862    0.0
35304    0.0
26138    0.0
52327    0.0
6038     0.0
        ... 
20609    0.0
21440    1.0
73349    0.0
50057    0.0
5192     1.0
Name: SoldFlag, Length: 75996, dtype: float64

## Build Pipeline

In [26]:
def build_pipeline():
    binary_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(drop='if_binary'))
    ])
    
    nominal_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    preprocessor = ColumnTransformer(transformers=[
        ('binary', binary_transformer, ['MarketingType']),
        ('nominal', nominal_transformer, ['ReleaseNumber'])
    ], remainder='passthrough')
    
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', RandomForestClassifier(random_state=1))
    ])
    
    return model

## Train/Validate

In [27]:
accs = []
f1s = []

kf = KFold(n_splits=5)

for train_idx, test_idx in kf.split(X):
    X_train = X.iloc[train_idx, :]
    X_test = X.iloc[test_idx, :]
    y_train = y.iloc[train_idx]
    y_test = y.iloc[test_idx]
    
    # Address class imbalance
    num_samples = int(y_train.value_counts().mean())
    majority_indices = y_train[y_train == 0.0].index
    samples_to_drop = y_train[majority_indices].sample(len(y_train) - num_samples, random_state=1).index
    X_train = X_train.drop(samples_to_drop, axis=0)
    y_train = y_train.drop(samples_to_drop, axis=0)
    oversampler = RandomOverSampler(random_state=1)
    X_train, y_train = oversampler.fit_resample(X_train, y_train)
    
    model = build_pipeline()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    accs.append(accuracy_score(y_test, y_pred))
    f1s.append(f1_score(y_test, y_pred, pos_label=1.0))

acc = np.mean(accs)
f1 = np.mean(f1s)

print("Accuracy: {:.2f}%".format(acc * 100))
print("F1-Score: {:.5f}".format(f1))

Accuracy: 76.87%
F1-Score: 0.43494
